LAGI DI RUN JERRY :)

In [ ]:
!pip install twint
!pip install sastrawi
!pip install keras
!pip freeze
!pip install nest-asyncio

In [ ]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style = 'whitegrid')

import twint
import nest_asyncio
nest_asyncio.apply()


import datetime as dt
import re
import string
          from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
from  tensorflow.keras.optimizers import Adam, RMSprop, SGD

from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Address_Update/clean_data/dataUse.csv')
df = df[0:50000]

In [ ]:
df

,street1,Codex,full_lower
0,Gang Veteran RT 01 RW 07 No.9 \nKelurahan Lub...,0,gang veteran rt 01 rw 07 no9 kelurahan lubang ...
1,Lingkungan TUMALIA kecamatan Turikale Keluraha...,1,lingkungan tumalia kecamatan turikale keluraha...
2,ds kalimalang dsn sragi lor rt03 rw02,2,ds kalimalang dsn sragi lor rt03 rw02
3,Desa Gunung Tandla rt 01 Rw 01\nkec. KAwalu\nk...,3,desa gunung tandla rt 01 rw 01 kec kawalu kota...
4,Ruko Mall Taman Palem No.70,4,ruko mall taman palem no70
...,...,...,...
49995,desa Taba teret (posisi rmh kami pinggir jala...,4509,desa taba teret posisi rmh kami pinggir jalan ...
49996,Jl. Nawawi gelar dalam sumberjo LK 0 Rt 001/Rw...,413,jl nawawi gelar dalam sumberjo lk 0 rt 001rw 0...
49997,Jln.Mertajaya IV B No.26 -80119\nKec.Denpasar...,320,jlnmertajaya iv b no26 80119 kecdenpasar barat...
49998,"Bukit cimanggu city blok S1/8 bogor, Tanah Sar...",162,bukit cimanggu city blok s18 bogor tanah sarea...


In [ ]:
# drop data ke- ([87,171,758,2070,2144,2355,2789,2889, 9, 19, 60, 100, 104, 133, 152, 159, 167, 172, 190, 275, 516, 827, 845, 870, 917, 944, 1000, 1001,
# 1003, 1516, 1693, 2014, 2054, 2066, 2082, 2051, 2114, 2123, 2340, 2313, 2421, 2542, 2530, 2699, 2761, 2746, 2903, 
# 3093, 3052, 3040, 3227, 3593, 3633, 3674, 3684, 3779, 3716, 3875, 3871, 3826, 3988 ,3972])

# ada catatan dari user, di drop aja. (contoh : nomer telepon user, patokan jalan, catatan barang yang dibeli(warna atau ukuran barang))

In [ ]:
#check data
df.iloc[[3040]]

,street1,Codex,full_lower
3040,"Jl. Cipaku IV No.1 RT.005/RW.05, Kelurahan Pet...",340,jl cipaku iv no1 rt005rw05 kelurahan petogogoa...


In [ ]:
df = df.drop(df.index[[87, 171, 758, 2070, 2144, 2355, 2789, 2889, 9, 19, 60, 100, 104, 133, 152, 159, 
                  167, 172, 190, 275, 516, 827, 845, 870, 917, 944, 1000, 1001, 1003, 1516, 1693, 
                  2014, 2054, 2066, 2082, 2051, 2114, 2123, 2340, 2313, 2421, 542, 2530, 2699, 2761, 
                  2746, 2903, 3093, 3052, 3040, 3227, 3593, 3633, 3674, 3684, 3779, 3716, 3875, 3871, 3826, 3988 ,3972]])

In [ ]:
df = df.reset_index(drop=True)
df

,street1,Codex,full_lower
0,Gang Veteran RT 01 RW 07 No.9 \nKelurahan Lub...,0,gang veteran rt 01 rw 07 no9 kelurahan lubang ...
1,Lingkungan TUMALIA kecamatan Turikale Keluraha...,1,lingkungan tumalia kecamatan turikale keluraha...
2,ds kalimalang dsn sragi lor rt03 rw02,2,ds kalimalang dsn sragi lor rt03 rw02
3,Desa Gunung Tandla rt 01 Rw 01\nkec. KAwalu\nk...,3,desa gunung tandla rt 01 rw 01 kec kawalu kota...
4,Ruko Mall Taman Palem No.70,4,ruko mall taman palem no70
...,...,...,...
49933,desa Taba teret (posisi rmh kami pinggir jala...,4509,desa taba teret posisi rmh kami pinggir jalan ...
49934,Jl. Nawawi gelar dalam sumberjo LK 0 Rt 001/Rw...,413,jl nawawi gelar dalam sumberjo lk 0 rt 001rw 0...
49935,Jln.Mertajaya IV B No.26 -80119\nKec.Denpasar...,320,jlnmertajaya iv b no26 80119 kecdenpasar barat...
49936,"Bukit cimanggu city blok S1/8 bogor, Tanah Sar...",162,bukit cimanggu city blok s18 bogor tanah sarea...


In [ ]:
custom_word = ['jln', 'jalan', 'jl', 'jln', 'gg', 'gang', 'no', 'nomor', 'ds', 'desa', 'dsn', 'rt', 'rw', 'kecamatan', 'kec', 'jln.','kelurahan','kl.']

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub('(\d+(\.\d+)?)', r' \1 ', text).strip() #add space between number and string in a sentence
    text = re.sub(r'(\w)(\1{2,})', r"\1", text) #will replace repated char
    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def filteringText(text): # Remove stopwors in a text
    #stopword
    text_split = text.split(' ')
    temp_text_split = []
    for i in range(len(text_split)):
      if (text_split[i] not in custom_word):
        temp_text_split.append(text_split[i])
    text = ' '.join(temp_text_split)
    return text

#PreProcessing
def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

In [ ]:
df['text_clean'] = df['street1'].apply(cleaningText)
df['text_clean'] = df['text_clean'].apply(casefoldingText)
df['text_clean'] = df['text_clean'].apply(filteringText)
df.drop(['full_lower'], axis = 1, inplace = True)

df['text_preprocessed'] = df['text_clean'].apply(tokenizingText)
df['text_preprocessed'] = df['text_preprocessed'].apply(stemmingText)

In [ ]:
df.rename(columns={'text_clean': 'full_lower'}, inplace=True)

In [ ]:
df

,street1,Codex,full_lower,text_preprocessed
0,Gang Veteran RT 01 RW 07 No.9 \nKelurahan Lub...,0,veteran 01 07 9 lubang buaya cipayung,"[veteran, 01, 07, 9, lubang, buaya, cipayung]"
1,Lingkungan TUMALIA kecamatan Turikale Keluraha...,1,lingkungan tumalia turikale adatongeng,"[lingkung, tumalia, turikale, adatongeng]"
2,ds kalimalang dsn sragi lor rt03 rw02,2,kalimalang sragi lor 03 02,"[kalimalang, sragi, lor, 03, 02]"
3,Desa Gunung Tandla rt 01 Rw 01\nkec. KAwalu\nk...,3,gunung tandla 01 01 kawalu kota tasikmalay...,"[gunung, tandla, 01, 01, kawalu, kota, tasikma..."
4,Ruko Mall Taman Palem No.70,4,ruko mall taman palem 70,"[ruko, mall, taman, palem, 70]"
...,...,...,...,...
49933,desa Taba teret (posisi rmh kami pinggir jala...,4509,taba teret posisi rmh kami pinggir lintas taba...,"[taba, teret, posisi, rmh, kami, pinggir, lint..."
49934,Jl. Nawawi gelar dalam sumberjo LK 0 Rt 001/Rw...,413,nawawi gelar dalam sumberjo lk 0 001 0 ra...,"[nawawi, gelar, dalam, sumberjo, lk, 0, 001, 0..."
49935,Jln.Mertajaya IV B No.26 -80119\nKec.Denpasar...,320,jlnmertajaya iv b 26 80119 kecdenpasar bar...,"[jlnmertajaya, iv, b, 26, 80119, kecdenpasar, ..."
49936,"Bukit cimanggu city blok S1/8 bogor, Tanah Sar...",162,bukit cimanggu city blok s 1 8 bogor tanah s...,"[bukit, cimanggu, city, blok, s, 1, 8, bogor, ..."


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Address_Update/clean_data/dataClean_1_revisi.csv', index=False)

In [ ]:
print('done')

done


In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Address_Update/clean_data/dataClean_1.csv')